In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from tqdm.auto import tqdm
from sklearn.model_selection import KFold

%matplotlib inline

In [80]:
df = pd.read_csv('../data/lesson3.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [81]:
df.columns = df.columns.str.lower().str.replace(' ','_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in categorical_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype('int')

In [82]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']
categorical = ['seniorcitizen','gender','partner','dependents','phoneservice','multiplelines','internetservice','onlinesecurity','onlinebackup','deviceprotection','techsupport','streamingtv','streamingmovies','contract','paperlessbilling','paymentmethod']

In [83]:
def train(df, y, c=1.0):
    dicts = df[categorical + numerical].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(max_iter=1000, C=c)
    model.fit(X_train, y)

    return dv, model

def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')
    X = dv.fit_transform(dicts)
    y_pred = model.predict_proba(X)[:,1]

    return y_pred

In [84]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_full_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

del df_train['churn']
del df_val['churn']
del df_test['churn']

In [85]:
def kfold_scores(kfold, df_full_train, c):
    scores = []
    for train_idxs, val_idxs in kfold.split(df_full_train):
        df_train = df_full_train.iloc[train_idxs]
        df_val = df_full_train.iloc[val_idxs]

        y_train = df_train.churn.values
        y_val = df_val.churn.values

        dv, model = train(df_train, y_train, c)
        y_pred = predict(df_val, dv, model)

        scores.append(roc_auc_score(y_val, y_pred))
    return scores

In [86]:
n_split = 5
kfold = KFold(n_splits=n_split, shuffle=True, random_state=1)
for c in tqdm([0.01, 0.1, 0.5, 1, 10]):
    scores = kfold_scores(kfold, df_full_train, c)
    print('%s -> %.3f +/- %.3f' % (c, np.mean(scores), np.std(scores)))


  0%|          | 0/5 [00:00<?, ?it/s]

0.01 -> 0.836 +/- 0.007
0.1 -> 0.839 +/- 0.008
0.5 -> 0.839 +/- 0.008
1 -> 0.839 +/- 0.008
10 -> 0.839 +/- 0.007


In [87]:
c = 0.5
y_full_train = df_full_train.churn.values
del df_full_train['churn']

In [88]:
dv, model = train(df_full_train, y_train, c)
y_pred = predict(df_test, dv, model)
auc = roc_auc_score(y_test, y_pred)
auc


0.8570056983760711

In [89]:
import pickle


In [90]:
output_file = f'model_C={c}.bin'
with open(output_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)


In [91]:
with open(output_file, 'rb') as f_in:
    dv_in, model_in = pickle.load(f_in)


In [99]:
customer = {'customerid': '5868-ytyks',
            'gender': 'male',
            'seniorcitizen': 0,
            'partner': 'no',
            'dependents': 'yes',
            'tenure': 10,
            'phoneservice': 'yes',
            'multiplelines': 'no',
            'internetservice': 'no',
            'onlinesecurity': 'no_internet_service',
            'onlinebackup': 'no_internet_service',
            'deviceprotection': 'no_internet_service',
            'techsupport': 'no_internet_service',
            'streamingtv': 'no_internet_service',
            'streamingmovies': 'no_internet_service',
            'contract': 'month-to-month',
            'paperlessbilling': 'yes',
            'paymentmethod': 'mailed_check',
            'monthlycharges': 20.25,
            'totalcharges': 20.25}

In [100]:
X = dv_in.transform(customer)
model_in.predict_proba(X)[0,1]


0.16275729579774528